# **Reproducing M1 results (Table 2)**

<a target="_blank" href="https://colab.research.google.com/github/blackswan-advitamaeternam/HVAE/blob/raph/paper_experiments/Table2_exp.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>

## **Colab setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

In [ ]:
!git clone https://github.com/blackswan-advitamaeternam/HVAE.git
%cd HVAE
!git checkout raph
!pip install -r requirements.txt

To allow automatic reloading

In [ ]:
!pip install --upgrade ipython

In [ ]:
import sys
try:
    import imp
except ImportError:
    import types
    sys.modules['imp'] = types.ModuleType('imp')

In [ ]:
%load_ext autoreload
%autoreload 2

## **Imports**

In [ ]:
import sys, os
# To ensure the custom package is found
path_to_repo = "/content/HVAE"
if path_to_repo not in sys.path:
    sys.path.append(path_to_repo)

In [ ]:
import numpy as np
import pandas as pd 
import torch
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

from svae.vae import M1, predict_classes_loader
from svae.training import training_M1
from svae.utils import ShuffledLoader

from paper_experiments.load_MNIST import make_splits_loaders_MNIST

Setting device

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# make splits
NUM_WORKERS = int(0.8*os.cpu_count())
FRAC = 0.1
TRAIN_FRAC = int(50000 * FRAC)
VAL_FRAC = int(10000 * FRAC)
TEST_FRAC = None # potentially still test on all test samples
print(f"Using {NUM_WORKERS} workers.")
train_loader, val_loader, test_loader = make_splits_loaders_MNIST(train_size=TRAIN_FRAC, val_size=VAL_FRAC, test_size=TEST_FRAC,
                                                                batch_size=64,
                                                                test_batch_size=100,
                                                                num_workers=NUM_WORKERS,
                                                                prefetch_factor=2,
                                                                force=True,
                                                                persistent_workers=True,
                                                                pin_memory=True)

In [ ]:
# Manifesting to memory of the device (we can afford it on colab ?)
train_batches = [[el.to(DEVICE) for el in batch] for batch in train_loader]
val_batches   = [[el.to(DEVICE) for el in batch] for batch in val_loader]
test_batches  = [[el.to(DEVICE) for el in batch] for batch in test_loader]

# Wrap for shuffling behavior
train_loader = ShuffledLoader(
    train_batches,
    shuffle_batches=True,
    shuffle_within_batch=True,
    device_for_randperm=DEVICE)

val_loader = val_batches
test_loader = test_batches

## **Configuration**

In [ ]:
base_path = "/content/drive/MyDrive/HVAE/Table2/"
os.makedirs(base_path, exist_ok=True)

In [ ]:
EPOCHS = 500
INPUT_DIM = 784
HIDDEN_DIM = 128

LATENT_MODE = 'sample'
PATIENCE = 50
WARMUP = 100
ONE_LAYER = False
LR = 1e-3

N_RUNS = 5

In [ ]:
def run_and_test(mode, latent, test_dataloader):
    addon = "[SVAE]" if mode == "svae" else "[NVAE]"

    print(f"\n{addon} Instantiating SVAE and optimizer..")
    model = M1(mode,
                INPUT_DIM,
                HIDDEN_DIM,
                latent,
                ONE_LAYER,
                )
    # To device
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    print(f"{addon} Started training..")
    model, losses, all_parts = training_M1(train_loader, 
                                    val_loader,
                                    model,
                                    optimizer,
                                    epochs=EPOCHS,
                                    beta_kl=1,
                                    warmup=WARMUP,
                                    patience=PATIENCE,
                                    show_loss_every=1,
                                    mode=LATENT_MODE)

    print(f"\n{addon} Predicting classes..")
    Y, Y_hat = predict_classes_loader(model, test_dataloader, LATENT_MODE)

    test_acc = accuracy_score(Y, Y_hat)
    print(f"{addon} Test accuracy: {test_acc*100:.2f}")
    return test_acc

In [ ]:
def launch_experiment(N, test_dataloader):
    results_df = {"Latent Size":[], "Model":[], "Accuracy":[], "Std":[], "N_test":[]}
    for latent in tqdm([2, 5, 10, 20, 40], desc="Exploring latent.."):
        for mode in ["svae", "normal"]:
            accuracy_list = []
            for i in tqdm(list(range(N_RUNS)), desc="Repeated runs.."):
                print(f"\nSTARTING RUN n°{i+1}")
                accuracy_list.append(run_and_test(mode, latent, test_dataloader))
            avg_acc = np.nanmean(accuracy_list)
            std_acc = np.nanstd(accuracy_list)

            results_df["Latent Size"].append(latent)
            results_df["Model"].append(mode)
            results_df["Accuracy"].append(avg_acc)
            results_df["Std"].append(std_acc)
            results_df["N_test"].append(N)
    return pd.DataFrame(results_df)

## **$N = 100$**
Test Batch size is 100 so only test on 1 batch

In [ ]:
sub_test_loader = [batch for i, batch in enumerate(test_loader) if i < 1]

In [ ]:
results_df_100 = launch_experiment(100, sub_test_loader)

In [ ]:
results_df_100.to_csv(base_path + "100_M1_results.csv")
results_df_100

## **$N = 600$**
Test Batch size is 100 so only test on 6 batches

In [ ]:
sub_test_loader = [batch for i, batch in enumerate(test_loader) if i < 6]

In [ ]:
results_df_600 = launch_experiment(600, sub_test_loader)

In [ ]:
results_df_600.to_csv(base_path + "600_M1_results.csv")
results_df_600

## **$N = 1000$**
only test on 10 batches

In [ ]:
sub_test_loader = [batch for i, batch in enumerate(test_loader) if i < 10]

In [ ]:
results_df_1000 = launch_experiment(1000, sub_test_loader)

In [ ]:
results_df_1000.to_csv(base_path + "1000_M1_results.csv")
results_df_1000